In [81]:
import read_data as imp
import numpy as np
import pandas as pd
import pyfolio as pf
import matplotlib.pyplot as plt
import Alphas101 as Alphas

import calendar

import itertools as it

import warnings
warnings.filterwarnings("ignore")

In [82]:
def set_date(year_month,day):
    return str(year_month[0])+'-'+str(year_month[1])+'-'+str(day)

symbols = list(pd.read_csv("Tickers.txt", sep="\n", header=None)[0].values)
symbols = ['XLP','XLY','XLE','XLF','XLV','XLI','XLK','XLU']

path_features = "Features.txt"
features = list(pd.read_csv(path_features, sep="\n", header=None)[0].values)

In [83]:
years = [x for x in range(2000,2020)]
months = [x+1 for x in range(12)]
years_months = list(it.product(*[years,months]))
ld_month = [calendar.monthrange(x[0],x[1])[1] for x in years_months]
Rebalancing_dates = pd.to_datetime([y for y in map(set_date, years_months, ld_month)])
Rebalancing_dates

DatetimeIndex(['2000-01-31', '2000-02-29', '2000-03-31', '2000-04-30',
               '2000-05-31', '2000-06-30', '2000-07-31', '2000-08-31',
               '2000-09-30', '2000-10-31',
               ...
               '2019-03-31', '2019-04-30', '2019-05-31', '2019-06-30',
               '2019-07-31', '2019-08-31', '2019-09-30', '2019-10-31',
               '2019-11-30', '2019-12-31'],
              dtype='datetime64[ns]', length=240, freq=None)

In [84]:
hist_data = imp.organizarTodo(symbols,years)
hist_feature = imp.organizarTodo(features,years,True)

In [15]:
for etf,data in hist_data.items():
    

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices